<a href="https://colab.research.google.com/github/emely3h/Geospatial_ML/blob/feature%2Fjaccard_index/data_exploration/physics_jaccard_emely.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Physics Jaccard Index

Calculate the physics jaccard index which will be the main success metric.

### 0. Prepare Colab

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
! ls
%cd drive/MyDrive/MachineLearning/Geospatial_ML
! ls

architecture.drawio  experiment_1_2.ipynb  prepare_data
data_exploration     experiments	   README.md
evaluation	     models		   requirements.txt
[Errno 2] No such file or directory: 'drive/MyDrive/MachineLearning/Geospatial_ML'
/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning/Geospatial_ML
architecture.drawio  experiment_1_2.ipynb  prepare_data
data_exploration     experiments	   README.md
evaluation	     models		   requirements.txt


In [8]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from data_exploration.mask_stats import Mask_Stats
from prepare_data.create_mask import create_physical_mask
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from datetime import datetime
from typing import Tuple

### 1. Helper functions

In [9]:
import numpy as np
from typing import Tuple
from keras.utils import to_categorical
from tensorflow import keras
from prepare_data.create_mask import create_physical_mask


class JaccardIndexCalculator:

    __slots__ = [
        "split_x",
        "split_y",
        "tiles",
        "num_classes",
        "chunk_size",
        "num_chunks",
        "current_chunk_index",
        "all_intersections",
        "all_unions",
        "each_intersection",
        "each_union",
        "each_jaccard_index",
    ]

    def __init__(
        self,
        split_x: np.memmap,
        split_y: np.memmap,
        tiles: int,
        num_classes: int = 3,
        chunk_size: int = 1000,
    ):
        self.split_x = split_x
        self.split_y = split_y
        self.tiles = tiles
        self.num_classes = num_classes
        self.chunk_size = chunk_size
        self.num_chunks = int(np.ceil(tiles / chunk_size))
        self.current_chunk_index = 0
        self.all_intersections = 0
        self.all_unions = 0
        self.each_intersection = np.zeros(num_classes + 1)
        self.each_union = np.zeros(num_classes + 1)
        self.each_jaccard_index = np.zeros(num_classes + 1)

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_chunk_index >= self.num_chunks:
            raise StopIteration
        print(f"Chunk No.{self.current_chunk_index}")
        print("Step1: Copying memmap to array")
        x_input_chunk, y_mask_chunk = self.copy_data_to_array()
        print("Step2: Creating Physical Mask")
        pred_physical = create_physical_mask(x_input_chunk)
        print("Step3: Hot Encoding")
        y_true = to_categorical(y_mask_chunk, num_classes=self.num_classes)
        print("Step4: Calculate Intersection and Union")
        self.intersection_union(y_true=y_true, y_pred=pred_physical)

        print("\n")
        self.current_chunk_index += 1

    def copy_data_to_array(self) -> Tuple[np.ndarray, np.ndarray]:
        start_index = self.current_chunk_index * self.chunk_size
        end_index = start_index + self.chunk_size
        if end_index > self.tiles:
            end_index = self.tiles
        chunk_size = end_index - start_index
        print("chunk_size:", chunk_size)
        x_input = np.zeros((chunk_size, 256, 256, 5), dtype=np.float32)
        np.copyto(x_input, self.split_x[start_index:end_index])
        y_mask = np.zeros((chunk_size, 256, 256), dtype=np.float32)
        np.copyto(y_mask, self.split_y[start_index:end_index])
        return x_input, y_mask

    def intersection_union(self, y_true: np.ndarray, y_pred: np.ndarray) -> None:
        for i in range(self.num_classes + 1):
            if not i == 3:
                y_true_f = keras.backend.flatten(y_true[..., i])
                y_pred_f = keras.backend.flatten(y_pred[..., i])
            else:
                y_true_f = keras.backend.flatten(y_true)
                y_pred_f = keras.backend.flatten(y_pred)
            intersection = keras.backend.sum(y_true_f * y_pred_f)
            union = (
                keras.backend.sum(y_true_f) + keras.backend.sum(y_pred_f) - intersection
            )
            print("class: ", i)
            print("intersection: ", intersection)
            print("union: ", union)

            self.each_intersection[i] += intersection
            self.each_union[i] += union
            if not i == 3:
                self.all_intersections += intersection
                self.all_unions += union
        return

    def calculate_mean_jaccard_index(self) -> dict:
        for num_chunks in self:
            pass
        print("Summary \n")
        print("total tiles: ", self.tiles)
        print("chunk size: ", self.chunk_size)

        for i in range(self.num_classes + 1):
            class_name = ["invalid", "valid", "land", "all_original"]
            print("class name: ", class_name[i])
            print("each_intersection: ", self.each_intersection[i])
            print("each_union: ", self.each_union[i])
            self.each_jaccard_index[i] = (self.each_intersection[i] + 1.0) / (
                self.each_union[i] + 1.0
            )
            print("each_jaccard_index: ", self.each_jaccard_index[i])

        print("total of intersections: ", self.all_intersections)
        print("total of unions: ", self.all_unions)
        all_mean_jaccard = (self.all_intersections + 1.0) / (self.all_unions + 1.0)
        print("mean of Jaccard Index: ", all_mean_jaccard)

        return {
            "mean_jaccard_sum": all_mean_jaccard,
            class_name[0]: self.each_jaccard_index[0],
            class_name[1]: self.each_jaccard_index[1],
            class_name[2]: self.each_jaccard_index[2],
            class_name[3]: self.each_jaccard_index[3],
        }

### 2. Jaccard index for overlapping tiles

In [10]:
data_path = "../data_colab/256_200"
train_tiles = 11063
val_tiles = 3545
test_tiles = 3699

train_split_x = np.memmap(os.path.join(data_path, "train_split_x.npy"), mode="r", shape=(train_tiles, 256, 256, 5), dtype=np.uint8)
train_split_y = np.memmap(os.path.join(data_path, "train_split_y.npy"), mode="r", shape=(train_tiles, 256, 256), dtype=np.uint8)
val_split_x = np.memmap(os.path.join(data_path, "val_split_x.npy"), mode="r", shape=(val_tiles, 256, 256, 5), dtype=np.uint8)
val_split_y = np.memmap(os.path.join(data_path, "val_split_y.npy"), mode="r", shape=(val_tiles, 256, 256), dtype=np.uint8)
test_split_x = np.memmap(os.path.join(data_path, "test_split_x.npy"), mode="r", shape=(test_tiles, 256, 256, 5), dtype=np.uint8)
test_split_y = np.memmap(os.path.join(data_path, "test_split_y.npy"), mode="r", shape=(test_tiles, 256, 256), dtype=np.uint8)

train_stats = Mask_Stats(train_split_y)
train_stats.print_stats()
print()
val_stats = Mask_Stats(val_split_y)
val_stats.print_stats()
print()
test_stats = Mask_Stats(test_split_y)
test_stats.print_stats()

Shape: (11063, 256, 256)
Land pixels: 326666615  45.056 %
Valid pixels: 231026701  31.865 %
Invalid pixels: 167331452  23.079 %
Sum: 11063

Shape: (3545, 256, 256)
Land pixels: 100682317  43.337 %
Valid pixels: 76811432  33.062 %
Invalid pixels: 54831371  23.601 %
Sum: 3545

Shape: (3699, 256, 256)
Land pixels: 112712687  46.495 %
Valid pixels: 71301683  29.413 %
Invalid pixels: 58403294  24.092 %
Sum: 3699


In [11]:
start = datetime.now()
print("Calculate Intersection and Union for training set \n")
train_jaccard_calc = JaccardIndexCalculator(train_split_x, train_split_y, train_tiles, chunk_size=1000)
train_jaccard_ = train_jaccard_calc.calculate_mean_jaccard_index()
print(train_jaccard_)

print("\n\nCalculate Intersection and Union for validation set \n")
val_jaccard_calc = JaccardIndexCalculator(val_split_x, val_split_y, val_tiles, chunk_size=1000)
val_jaccard_ = val_jaccard_calc.calculate_mean_jaccard_index()
print(train_jaccard_)

print("\n\nCalculate Intersection and Union for testing set \n")
test_jaccard_calc = JaccardIndexCalculator(test_split_x, test_split_y, test_tiles, chunk_size=1000)
test_jaccard_ = test_jaccard_calc.calculate_mean_jaccard_index()
print(train_jaccard_)

end = datetime.now()
print(f'time needed: {end - start}')

Calculate Intersection and Union for training set 

Chunk No.0
Step1: Copying memmap to array
chunk_size: 1000
Step2: Creating Physical Mask
Step3: Hot Encoding
Step4: Calculate Intersection and Union
class:  0
intersection:  tf.Tensor(2094040.0, shape=(), dtype=float32)
union:  tf.Tensor(4119007.0, shape=(), dtype=float32)
class:  1
intersection:  tf.Tensor(30360342.0, shape=(), dtype=float32)
union:  tf.Tensor(32385306.0, shape=(), dtype=float32)
class:  2
intersection:  tf.Tensor(31056652.0, shape=(), dtype=float32)
union:  tf.Tensor(31056652.0, shape=(), dtype=float32)
class:  3
intersection:  tf.Tensor(63511030.0, shape=(), dtype=float32)
union:  tf.Tensor(67560960.0, shape=(), dtype=float32)


Chunk No.1
Step1: Copying memmap to array
chunk_size: 1000
Step2: Creating Physical Mask
Step3: Hot Encoding
Step4: Calculate Intersection and Union
class:  0
intersection:  tf.Tensor(3616129.0, shape=(), dtype=float32)
union:  tf.Tensor(5502978.0, shape=(), dtype=float32)
class:  1
interse

KeyboardInterrupt: ignored

### 3. Jaccard index for non-overlapping tiles

In [5]:
total_tiles = 11121
train_tiles = total_tiles // 100 * 60 +1
test_val_tiles = total_tiles // 100 * 20 +1
data_path = "../data_colab/256_256"

train_split_x = np.memmap(os.path.join(data_path, "train_split_x.npy"), mode="r", shape=(train_tiles, 256, 256, 5), dtype=np.float32)
train_split_y = np.memmap(os.path.join(data_path, "train_split_y.npy"), mode="r", shape=(train_tiles, 256, 256), dtype=np.float32)
val_split_x = np.memmap(os.path.join(data_path, "val_split_x.npy"), mode="r", shape=(test_val_tiles, 256, 256, 5), dtype=np.float32)
val_split_y = np.memmap(os.path.join(data_path, "val_split_y.npy"), mode="r", shape=(test_val_tiles, 256, 256), dtype=np.float32)
test_split_x = np.memmap(os.path.join(data_path, "test_split_x.npy"), mode="r", shape=(test_val_tiles, 256, 256, 5), dtype=np.float32)
test_split_y = np.memmap(os.path.join(data_path, "test_split_y.npy"), mode="r", shape=(test_val_tiles, 256, 256), dtype=np.float32)

train_stats = Mask_Stats(train_split_y)
train_stats.print_stats()
print()
val_stats = Mask_Stats(val_split_y)
val_stats.print_stats()
print()
test_stats = Mask_Stats(test_split_y)
test_stats.print_stats()

Shape: (6661, 256, 256)
Land pixels: 176919986  40.528 %
Valid pixels: 125877821  28.836 %
Invalid pixels: 133737489  30.636 %
Sum: 6661

Shape: (2221, 256, 256)
Land pixels: 59840780  41.112 %
Valid pixels: 41275933  28.358 %
Invalid pixels: 44438743  30.530 %
Sum: 2221

Shape: (2221, 256, 256)
Land pixels: 59010175  40.541 %
Valid pixels: 42215169  29.003 %
Invalid pixels: 44330112  30.456 %
Sum: 2221


In [6]:
start = datetime.now()
print("Calculate Intersection and Union for training set \n")
train_jaccard_calc = JaccardIndexCalculator(train_split_x, train_split_y, train_tiles, chunk_size=1000)
train_jaccard_ = train_jaccard_calc.calculate_mean_jaccard_index()
print(train_jaccard_)

print("\n\nCalculate Intersection and Union for validation set \n")
val_jaccard_calc = JaccardIndexCalculator(val_split_x, val_split_y, test_val_tiles, chunk_size=1000)
val_jaccard_ = val_jaccard_calc.calculate_mean_jaccard_index()
print(train_jaccard_)

print("\n\nCalculate Intersection and Union for testing set \n")
test_jaccard_calc = JaccardIndexCalculator(test_split_x, test_split_y, test_val_tiles, chunk_size=1000)
test_jaccard_ = test_jaccard_calc.calculate_mean_jaccard_index()
print(train_jaccard_)

end = datetime.now()
print(f'time needed: {end - start}')

Calculate Intersection and Union for training set 

Chunk No.0
Step1: Copying memmap to array
chunk_size: 1000
Step2: Creating Physical Mask
Step3: Hot Encoding
Step4: Calculate Intersection and Union
class:  0
intersection:  tf.Tensor(16969458.0, shape=(), dtype=float32)
union:  tf.Tensor(21134014.0, shape=(), dtype=float32)
class:  1
intersection:  tf.Tensor(17113096.0, shape=(), dtype=float32)
union:  tf.Tensor(21277652.0, shape=(), dtype=float32)
class:  2
intersection:  tf.Tensor(27288892.0, shape=(), dtype=float32)
union:  tf.Tensor(27288892.0, shape=(), dtype=float32)
class:  3
intersection:  tf.Tensor(61371444.0, shape=(), dtype=float32)
union:  tf.Tensor(69700560.0, shape=(), dtype=float32)


Chunk No.1
Step1: Copying memmap to array
chunk_size: 1000
Step2: Creating Physical Mask
Step3: Hot Encoding
Step4: Calculate Intersection and Union
class:  0
intersection:  tf.Tensor(12112116.0, shape=(), dtype=float32)
union:  tf.Tensor(14069936.0, shape=(), dtype=float32)
class:  1
int

KeyboardInterrupt: ignored

### Notes

In [ ]:
mean_jaccard_2 = get_mean_jaccard(6661)

total tiles: 6661 batches: 1 rest: 0

Calculating jaccard index for batch 0, copy tiles [0:6661] from mmap

x_input after copying batch from mmap

x_input shape: (6661, 256, 256, 5)
x_input max: 255.0
x_input min: 0.0

y_mask after copying batch from mmap

y_mask shape: (6661, 256, 256)
y_mask max: 2.0
y_mask min: 0.0
pred mask shape: (6661, 256, 256, 3)
true mask shape: (6661, 256, 256, 3)
jaccard index: 0.7695764899253845


when executed in batches (batch size ~ 500) no GPU, no additional System RAM needed

Time to execute: 2 min


In [ ]:
# todo: over entire dataset, not only training and both for non-overlapping and overlapping => should be almost equal
# todo: calculate index separately for each class

In [ ]:
class JaccardIndexCalculator:

    def __init__(self, split_x: np.memmap, split_y: np.memmap, tiles: int, chunk_size: int):
        self.split_x = split_x
        self.split_y = split_y
        self.tiles = tiles
        self.chunk_size = chunk_size
        self.num_chunks = int(np.ceil(tiles/chunk_size))
        self.current_chunk_index = 0
        self.intersection = 0
        self.union = 0

    def __iter__(self):
        return self

    def __next__(self):
       if self.current_chunk_index >= self.num_chunks:
            raise StopIteration
       print(f"Calculating intersection and union for chunk {self.current_chunk_index}")
       x_input_chunk, y_mask_chunk = self.copy_data_to_array()
       pred_physical = create_physical_mask(x_input_chunk)
       y_one_hot = to_categorical(y_mask_chunk, num_classes=3)
       inter_union = self.intersection_union(pred_physical[:, :, :, 1], y_one_hot[:, :, :, 1])
       
       print(f"Intersection:{inter_union[0]}, union: {inter_union[1]} \n")
       
       
       self.intersection += inter_union[0]
       self.union += inter_union[1]
  
       self.current_chunk_index += 1
       return inter_union


    def copy_data_to_array(self) -> Tuple[np.ndarray, np.ndarray]:
      start_index = self.current_chunk_index * self.chunk_size
      end_index = start_index + self.chunk_size
      if end_index > self.tiles:
          end_index = self.tiles
      chunk_size = end_index - start_index
      x_input = np.zeros((chunk_size, 256, 256, 5), dtype=np.float32)
      np.copyto(x_input, self.split_x[start_index:end_index])
      y_mask = np.zeros((chunk_size, 256, 256), dtype=np.float32)
      np.copyto(y_mask, self.split_y[start_index:end_index])
      print(f"Copyed from mmap [{start_index}:{end_index}]")
      return x_input, y_mask
    
    def intersection_union(self, y_true, y_pred):
      y_true_f = keras.backend.flatten(y_true)
      y_pred_f = keras.backend.flatten(y_pred)

      intersection = keras.backend.sum(y_true_f * y_pred_f)
      return (intersection, (keras.backend.sum(y_true_f) + keras.backend.sum(y_pred_f) - intersection))
    
    def mean_jaccard(self):
      for chunk in self:
        pass
      return [(self.intersection +1) / (self.union + 1), self.intersection, self.union]

test = JaccardIndexCalculator(train_split_x, train_split_y, train_tiles, 500)
start = datetime.now()
mean_jaccard = test.mean_jaccard()
end = datetime.now()
print(f'time needed: {end - start}')
print(mean_jaccard)